In [3]:
import pandas as pd
filename = '10m.txt'

In [16]:
# Читаємо файл частинами
chunksize = 10**6  # Зчитуємо по мільйону рядків за раз
max_num = 0
min_num = 0
total_sum = 0
total_count = 0
numbers = []


for chunk in pd.read_csv(filename, chunksize=chunksize, header=None):
    #Пишу такий перебір тільки через вимогу завдання
    #При перевірці виявив що код з стандартними функціями min(), max() майже у 2 рази швидше
    for i in chunk[0]:
        if i > max_num:
            max_num = i
        if i < min_num:
            min_num = i
        total_sum += i
        total_count += 1
        


Час виконання: 3.449308156967163 секунд


In [17]:
mean = total_sum / total_count

print("Max number:", max_num)
print("Min number:", min_num)
print("Mean:", mean)

Max number: 49999978
Min number: -49999996
Mean: 7364.418442641844


In [18]:
#Друга більш складна частина

In [19]:
def find_longest_sequence_in_chunk(numbers, last_num, longest, current, increasing=True):
    start, end = longest
    current_start = current[0] if current else 0
    current_len = len(current)
    
    for i, num in enumerate(numbers):
        if last_num is not None and ((last_num < num and increasing) or (last_num > num and not increasing)):
            current_len += 1
        else:
            if current_len > end - start:
                start, end = current_start, current_start + current_len
                current = numbers[start:end]  # Зберігаємо найдовшу послідовність
            current_start = i
            current_len = 1

        last_num = num

    if current_len > end - start:
        start, end = current_start, current_start + current_len
        current = numbers[start:end]  # Оновлюємо найдовшу послідовність, якщо поточна довша

    return (start, end), current_start, current_len, last_num, current

longest_inc = (0, 0)
current_inc = []
last_inc_num = None

longest_dec = (0, 0)
current_dec = []
last_dec_num = None

inc_sequence = []
dec_sequence = []

for chunk in pd.read_csv(filename, chunksize=chunksize, header=None):
    numbers = chunk[0].tolist()

    # Знаходимо найдовші зростаючі послідовності
    longest_inc, current_start, current_len, last_inc_num, current_inc = find_longest_sequence_in_chunk(
        numbers, last_inc_num, longest_inc, current_inc, increasing=True
    )
    if len(current_inc) > len(inc_sequence):
        inc_sequence = current_inc  # Оновлюємо найдовшу зростаючу послідовність

    # Знаходимо найдовші спадні послідовності
    longest_dec, current_start, current_len, last_dec_num, current_dec = find_longest_sequence_in_chunk(
        numbers, last_dec_num, longest_dec, current_dec, increasing=False
    )
    if len(current_dec) > len(dec_sequence):
        dec_sequence = current_dec  # Оновлюємо найдовшу спадну послідовність

print("Найдовша зростаюча послідовність:", inc_sequence)
print("Найдовша спадна послідовність:", dec_sequence)


Найдовша зростаюча послідовність: [-40353787, -38132552, -12387052, -6963165, -3314171, 4399082, 8225082, 25009364, 44410340]
Найдовша спадна послідовність: [46324925, 31502922, 20830571, 5981071, -29343213, -32979305, -34625920, -39672461, -42895914, -48468169]


In [20]:
#Узявши до уваги кількість чисел у кожному чанку вирішив що цей алгоритм сортування найкраще підходить для виконання завдання
def merge_sort(arr):
    if len(arr) > 1:
        mid = len(arr) // 2
        left_half = arr[:mid]
        right_half = arr[mid:]

        merge_sort(left_half)
        merge_sort(right_half)

        i = j = k = 0

        while i < len(left_half) and j < len(right_half):
            if left_half[i] < right_half[j]:
                arr[k] = left_half[i]
                i += 1
            else:
                arr[k] = right_half[j]
                j += 1
            k += 1

        while i < len(left_half):
            arr[k] = left_half[i]
            i += 1
            k += 1

        while j < len(right_half):
            arr[k] = right_half[j]
            j += 1
            k += 1

    return arr

In [24]:
#У цій частині окрім знаходження медіани хочу показати знання парсингу з файлу
#вище була задіяна бібліотека пандас для оптимізації часу виконання
# Відкрити файл з числами у режимі читання
with open(filename, 'r') as file:
    # Прочитати кожен рядок файлу, видалити зайві пробіли та перевести у список чисел
    numbers = [int(line.strip()) for line in file]

# Виклик функції для сортування списку чисел методом злиття (merge sort)
numbers = merge_sort(numbers)

# Функція для знаходження медіани у відсортованому списку чисел
def find_median(numbers):
    # Визначити кількість чисел у списку
    n = len(numbers)
    # Якщо кількість чисел парна
    if n % 2 == 0:
        # Медіана - це середнє значення двох середніх чисел
        median = (numbers[n//2 - 1] + numbers[n//2]) / 2
    else:
        # Якщо кількість чисел непарна, медіана - це середнє число у списку
        median = numbers[n//2]

    # Повернути значення медіани
    return median

# Виклик функції для знаходження медіани
median = find_median(numbers)
print(f"The median is: {median}")

The median is: 25216
